# Imports

In [40]:
%load_ext autoreload
%autoreload 2

import numpy as np

from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA

from sklearn.datasets import *

from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import copy

from data_structures.tree_classifier import TreeClassifier
import utils.utils
import time

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Prepare Data

In [1]:
# Download all datasets from sklearn
for m in [fetch_olivetti_faces, fetch_20newsgroups_vectorized, fetch_lfw_people, fetch_lfw_pairs, fetch_covtype, fetch_rcv1, fetch_kddcup99, fetch_california_housing]:
    print(m)
    try:
        all_ = m()
        train = m(subset='train')
        test = m(subset='test')
    except:
        pass

<function fetch_olivetti_faces at 0x7fb910b9f940>
<function fetch_20newsgroups_vectorized at 0x7fb910b71e50>
<function fetch_lfw_people at 0x7fb910af15e0>
<function fetch_lfw_pairs at 0x7fb910af1700>
<function fetch_covtype at 0x7fb910a9be50>
<function fetch_rcv1 at 0x7fb9009f19d0>
<function fetch_kddcup99 at 0x7fb910af1040>
<function fetch_california_housing at 0x7fb9009f1820>


In [2]:
# Download the data from two categories
cats = ['alt.atheism', 'sci.space']
ng_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'), categories=cats)
ng_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'), categories=cats)


vectorizer = TfidfVectorizer()
trans = vectorizer.fit(ng_train.data)
train_vectors = vectorizer.transform(ng_train.data)
test_vectors = vectorizer.transform(ng_test.data)
print("Number of datapoints: ", len(ng_train.data))
print("Number of features: ", train_vectors.shape[1])
print("Balance: ", np.sum(ng_train.target) / len(ng_train.target)) # 55-45, roughly balanced

N_COMPONENTS=100
pca = PCA(n_components=N_COMPONENTS)
pca.fit(train_vectors.toarray())
pca_train_vecs = pca.transform(train_vectors.toarray())
pca_test_vecs = pca.transform(test_vectors.toarray())

Number of datapoints:  1073
Number of features:  18217
Balance:  0.5526561043802423


# Compare our implementation's accuracy to sklearn

In [3]:
dt = DecisionTreeClassifier(random_state=0)
dt.fit(pca_train_vecs,ng_train.target)
print("sklearn Decision Tree Accuracy:", np.mean(dt.predict(pca_test_vecs) == ng_test.target))

#cross_val_score(dt, pca_train_vecs, ng_train.target, cv=10).mean()

sklearn Decision Tree Accuracy: 0.7868162692847125


In [4]:
rf = RandomForestClassifier(random_state=0)
rf.fit(pca_train_vecs,ng_train.target)
print("sklearn Random Forest Accuracy:", np.mean(rf.predict(pca_test_vecs) == ng_test.target))

#cross_val_score(rf, pca_train_vecs, ng_train.target, cv=10).mean()

sklearn Random Forest Accuracy: 0.7966339410939691


In [8]:
classes_arr = np.unique(ng_train.target)
classes = utils.utils.class_to_idx(classes_arr)

In [9]:
tc = TreeClassifier(data=pca_train_vecs, labels=ng_train.target, max_depth=5, classes=classes, verbose=False)
start = time.time()
tc.fit()
end = time.time()
print("Train accuracy:", np.mean(tc.predict_batch(pca_train_vecs)[0] == ng_train.target))
print("Test accuracy:", np.mean(tc.predict_batch(pca_test_vecs)[0] == ng_test.target))
print("Num queries:", tc.num_queries)
print("Runtime:", end-start)

Train accuracy: 0.869524697110904
Test accuracy: 0.791023842917251
Num queries: 9756
Runtime: 8.611066818237305


In [10]:
tc = TreeClassifier(data=pca_train_vecs, labels=ng_train.target, max_depth=5, classes=classes, solver="EXACT", verbose=False)
start = time.time()
tc.fit()
end = time.time()
print("Train accuracy:", np.mean(tc.predict_batch(pca_train_vecs)[0] == ng_train.target))
print("Test accuracy:", np.mean(tc.predict_batch(pca_test_vecs)[0] == ng_test.target))
print("Num queries:", tc.num_queries)
print("Runtime:", end-start)

Train accuracy: 0.8732525629077353
Test accuracy: 0.7896213183730715
Num queries: 5365
Runtime: 5.37089729309082


# Make the dataset huge

In [41]:
doublings = 4
pca_train_vecs_huge = copy.deepcopy(pca_train_vecs)
pca_train_labels_huge = copy.deepcopy(ng_train.target)
print(pca_train_vecs_huge.shape)
for i in range(doublings):
    pca_train_vecs_huge = np.concatenate((pca_train_vecs_huge, pca_train_vecs_huge))
    pca_train_labels_huge = np.concatenate((pca_train_labels_huge, pca_train_labels_huge))
print(pca_train_vecs_huge.shape)

(1073, 100)
(17168, 100)


In [42]:
tc = TreeClassifier(data=pca_train_vecs_huge, labels=pca_train_labels_huge, max_depth=2, classes=classes, verbose=True, random_state=0)
start = time.time()
tc.fit()
end = time.time()
print("Train accuracy:", np.mean(tc.predict_batch(pca_train_vecs)[0] == ng_train.target))
print("Test accuracy:", np.mean(tc.predict_batch(pca_test_vecs)[0] == ng_test.target))
print("Num queries:", tc.num_queries)
print("Runtime:", end-start)
tc.tree_print()

Calculated split with 2000 queries
Calculated split with 10728 queries
Calculated split with 22440 queries
Fitting finished
Train accuracy: 0.8070829450139795
Test accuracy: 0.761570827489481
Num queries: 35168
Runtime: 7.173810005187988
|--- feature_1 <= -0.03711226766268916
|   |--- feature_3 <= 0.11135030598655185
|   |   |--- class: 0
|   |--- feature_3 > 0.11135030598655185
|   |   |--- class: 1
|--- feature_1 > -0.03711226766268916
|   |--- feature_3 <= -0.07697387276886158
|   |   |--- class: 0
|   |--- feature_3 > -0.07697387276886158
|   |   |--- class: 1




In [44]:
# We should implement vanilla TreeClassifier --> which uses identity bins

tc = TreeClassifier(data=pca_train_vecs_huge, labels=pca_train_labels_huge, max_depth=2, classes=classes, solver="EXACT", verbose=True, random_state=0)
start = time.time()
tc.fit()
end = time.time()
print("Train accuracy:", np.mean(tc.predict_batch(pca_train_vecs)[0] == ng_train.target))
print("Test accuracy:", np.mean(tc.predict_batch(pca_test_vecs)[0] == ng_test.target))
print("Num queries:", tc.num_queries)
print("Runtime:", end-start)
tc.tree_print()

Calculated split with 17168 queries
Calculated split with 5728 queries
Calculated split with 11440 queries
Fitting finished
Train accuracy: 0.8070829450139795
Test accuracy: 0.761570827489481
Num queries: 34336
Runtime: 16.79027009010315
|--- feature_1 <= -0.03711226766268916
|   |--- feature_3 <= 0.11135030598655185
|   |   |--- class: 0
|   |--- feature_3 > 0.11135030598655185
|   |   |--- class: 1
|--- feature_1 > -0.03711226766268916
|   |--- feature_3 <= -0.07697387276886158
|   |   |--- class: 0
|   |--- feature_3 > -0.07697387276886158
|   |   |--- class: 1


